## Advent of Code Day 7

In [1]:
import pandas as pd
import re

Import File

In [66]:
with open("D7 Input.txt", "r") as file:
    file_contents = file.read()
# print(file_contents)
ls = file_contents.split("\n")
ls[:10]

['TT999 460',
 '96T99 361',
 'K264Q 948',
 'K8TT3 68',
 'T5JT5 730',
 '88789 928',
 '262A6 561',
 'AA5AA 673',
 '2AT22 968',
 '9492K 597']

Parse everything

In [67]:
hands = [x.split(' ') for x in ls]
hands[:5]

[['TT999', '460'],
 ['96T99', '361'],
 ['K264Q', '948'],
 ['K8TT3', '68'],
 ['T5JT5', '730']]

Create an ordering dict:

In [68]:
ordering_map = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, 'T': 8, 'J': 9, 'Q': 10, 'K': 11, 'A': 12}

Create a function to work out what thing a hand should be:

In [69]:
def score_a_hand(hand):
    hand_ls = [*hand]
    distinct_ls = list(set(hand_ls))
    cards_ls = []
    most_cards = 0
    hand_score = 0
    for card in distinct_ls:
        card_appearances = hand_ls.count(card)
        cards_ls.append([card, card_appearances])
        if card_appearances > most_cards:
            most_cards = card_appearances
    if most_cards == 5:
        hand_score = 7
    elif most_cards == 4:
        hand_score = 6
    elif most_cards == 3:
        if len(distinct_ls) == 2:
            hand_score = 5
        else:
            hand_score = 4
    elif most_cards == 2:
        if len(distinct_ls) == 3:
            hand_score = 3
        else:
            hand_score = 2
    elif most_cards == 1:
        hand_score = 1
    else:
        hand_score = 0
    return hand_score

Create a dataframe with each hand and its bid:

In [70]:
rankings = []

for hand in hands:
    hand_score = score_a_hand(hand[0])
    rankings.append([hand[0], hand_score, ordering_map[hand[0][0]], ordering_map[hand[0][1]], ordering_map[hand[0][2]], ordering_map[hand[0][3]], ordering_map[hand[0][4]], hand[1]])

rankings[:5]

[['TT999', 5, 8, 8, 7, 7, 7, '460'],
 ['96T99', 4, 7, 4, 8, 7, 7, '361'],
 ['K264Q', 1, 11, 0, 4, 2, 10, '948'],
 ['K8TT3', 2, 11, 6, 8, 8, 1, '68'],
 ['T5JT5', 3, 8, 3, 9, 8, 3, '730']]

In [82]:
df = pd.DataFrame(rankings, columns=['hand', 'hand_score', 'card1', 'card2', 'card3', 'card4', 'card5', 'bid'])
df = df.sort_values(by=['hand_score', 'card1', 'card2', 'card3', 'card4', 'card5'], ascending=True).reset_index(drop=True).reset_index()
df = df.rename(columns={'index': 'rank'})
df['rank'] = df['rank'] + 1
df['winnings'] = df.apply(lambda x: int(x['rank']) * int(x['bid']), axis=1)
total_winnings = df['winnings'].sum()
print(total_winnings)

246795406


Part 2:

This part is pretty confusing because Jacks suddenly act like Jokers, so they are allowed to be treated like the highest value thing for that hand.  let's make a function that cycles through all potential options to find the best one and then rescore.

In [72]:
def score_a_hand_jokers(hand):
    rank = 0
    for card_type in ordering_map.keys():
        new_hand = hand.replace('J', card_type)
        if score_a_hand(new_hand) > rank:
            rank = score_a_hand(new_hand)
    return rank

score_a_hand_jokers('T5JT5')

5

Now put this in our dataframe and see if it does any good!

Ah I need to rank J lower than the value of a 2.

In [81]:
ordering_map_v2 = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, 'T': 8, 'J': -1, 'Q': 10, 'K': 11, 'A': 12}

In [83]:
df['joker_hand_score'] = df.apply(lambda x: score_a_hand_jokers(x['hand']), axis=1)
df['joker_card_1'] = df.apply(lambda x: ordering_map_v2[x['hand'][0]], axis=1)
df['joker_card_2'] = df.apply(lambda x: ordering_map_v2[x['hand'][1]], axis=1)
df['joker_card_3'] = df.apply(lambda x: ordering_map_v2[x['hand'][2]], axis=1)
df['joker_card_4'] = df.apply(lambda x: ordering_map_v2[x['hand'][3]], axis=1)
df['joker_card_5'] = df.apply(lambda x: ordering_map_v2[x['hand'][4]], axis=1)

df = df.sort_values(by=['joker_hand_score', 'joker_card_1', 'joker_card_2', 'joker_card_3', 'joker_card_4', 'joker_card_5'], ascending=True).reset_index(drop=True).reset_index()
df = df.rename(columns={'index': 'joker_rank'})
df['joker_rank'] = df['joker_rank'] + 1
df['joker_winnings'] = df.apply(lambda x: int(x['joker_rank']) * int(x['bid']), axis=1)
df.tail()

,joker_rank,rank,hand,hand_score,card1,card2,card3,card4,card5,bid,winnings,joker_hand_score,joker_card_1,joker_card_2,joker_card_3,joker_card_4,joker_card_5,joker_winnings
995,996,972,TTJTT,6,8,8,9,8,8,950,923400,7,8,8,-1,8,8,946200
996,997,885,QJJJQ,5,10,9,9,9,10,299,264615,7,10,-1,-1,-1,10,298103
997,998,979,QJQQQ,6,10,9,10,10,10,677,662783,7,10,-1,10,10,10,675646
998,999,893,KJKKJ,5,11,9,11,11,9,850,759050,7,11,-1,11,11,-1,849150
999,1000,997,AAAJA,6,12,12,12,9,12,166,165502,7,12,12,12,-1,12,166000


In [84]:
total_winnings = df['joker_winnings'].sum()
print(total_winnings)

249356515
